# An introduction to Ordinary Differential Equations

<p class="pm-node nj-authors">Marc Sturrock</p>

Inspired by source: <https://github.com/JuliaDiffEq/DiffEqTutorials.jl>

# Ordinary Differential Equations (ODEs)

This notebook will get you started with DifferentialEquations.jl by introducing you to the functionality for solving ordinary differential equations (ODEs). The corresponding documentation page is the [ODE tutorial](https://docs.sciml.ai/DiffEqDocs/stable/getting_started/). While some of the syntax may be different for other types of equations, the same general principles hold in each case. The goal is to give a gentle and thorough introduction that highlights these principles in a way that will help you generalise what you have learned.

## Background

If you are new to the study of differential equations, it can be helpful to do a quick background read on [the definition of ordinary differential equations](https://en.wikipedia.org/wiki/Ordinary_differential_equation). We define an ordinary differential equation as an equation which describes the way that a variable $u$ changes, that is

$u' = f(u,p,t)$

where $p$ are the parameters of the model, $t$ is the time variable, and $f$ is the nonlinear model of how $u$ changes. The initial value problem also includes the information about the starting value:

$u(t_0) = u_0$

Together, if you know the starting value and you know how the value will change with time, then you know what the value will be at any time point in the future. This is the intuitive definition of a differential equation.

## First Model: Exponential Growth

Our first model will be the canonical exponential growth model. An exponential growth model can be used to model many natural phenomena, such as population growth in ecosystems, the spread of infectious diseases in a susceptible population, the accumulation of compound interest in finance, and the proliferation of cells in biology.  This model says that the rate of change is proportional to the current value, and is this:

$u' = au$

where we have a starting value $u(0)=u_0$. Let's say we put 1 dollar into Bitcoin which is increasing at a rate of $98%$ per year. Then calling now $t=0$ and measuring time in years, our model is:

$u' = 0.98u$

and $u(0) = 1.0$. We encode this into Julia by noticing that, in this setup, we match the general form when

In [ ]:
f(u,p,t) = 0.98u

with $u_0 = 1.0$. If we want to solve this model on a time span from `t=0.0` to `t=1.0`, then we define an `ODEProblem` by specifying this function `f`, this initial condition `u0`, and this time span as follows:

In [ ]:
using OrdinaryDiffEq
f(u,p,t) = 0.98u
u0 = 1.0
tspan = (0.0,1.0)
prob = ODEProblem(f,u0,tspan)

To solve our `ODEProblem` we use the command `solve`.

In [ ]:
sol = solve(prob, Tsit5())

and that's it: we have succesfully solved our first ODE!

## Analysing the Solution

Of course, the solution type is not interesting in and of itself. We want to understand the solution! The documentation page which explains in detail the functions for analyzing the `solution` is the [Solution Handling](https://docs.sciml.ai/DiffEqDocs/stable/basics/solution/) page. Here we will describe some of the basics. You can plot the solution using the plot recipe provided by [Plots.jl](http://docs.juliaplots.org/latest/):

In [ ]:
using Plots; gr(lw=3)
plot(sol)

From the picture we see that the solution is an exponential curve, which matches our intuition. As a plot recipe, we can annotate the result using any of the [Plots.jl attributes](http://docs.juliaplots.org/latest/attributes/). For example:

In [ ]:
plot(sol, linewidth=5,
     title="Solution to the linear ODE with a thick line",
     xaxis="Time (t)", yaxis="u(t) (in μm)",
     label="My Thick Line!") # legend=false

Using the mutating `plot!` command we can add other pieces to our plot. For this ODE we know that the true solution is $u(t) = u_0 exp(at)$, so let's add some of the true solution to our plot:

In [ ]:
plot!(sol.t, t->1.0*exp(0.98t), lw=3, ls=:dash, label="True Solution!")

In the previous command I demonstrated `sol.t`, which grabs the array of time points that the solution was saved at:

In [ ]:
sol.t

We can get the array of solution values using `sol.u`:

In [ ]:
sol.u

`sol.u[i]` is the value of the solution at time `sol.t[i]`. We can compute arrays of functions of the solution values using standard comprehensions, like:

In [ ]:
[t+u for (u,t) in tuples(sol)]

However, one interesting feature is that, by default, the solution is a continuous function. If we check the print out again:

In [ ]:
sol

you see that it says that the solution has a order changing interpolation. The default algorithm automatically switches between methods in order to handle all types of problems. For non-stiff equations (like the one we are solving), it is a continuous function of 4th order accuracy. We can call the solution as a function of time `sol(t)`. For example, to get the value at `t=0.45`, we can use the command:

In [ ]:
sol(0.45)

## Controlling the Solver

DifferentialEquations.jl has a common set of solver controls among its algorithms which can be found [at the Common Solver Options](http://docs.juliadiffeq.org/latest/basics/common_solver_opts.html) page. We will detail some of the most widely used options.

### Tolerances

The most useful options are the tolerances `abstol` and `reltol`. These tell the internal adaptive time stepping engine how precise of a solution you want. Generally, `reltol` is the relative accuracy while `abstol` is the accuracy when `u` is near zero. These tolerances are local tolerances and thus are not global guarantees. However, a good rule of thumb is that the total solution accuracy is 1-2 digits less than the relative tolerances. Thus for the defaults `abstol=1e-6` and `reltol=1e-3`, you can expect a global accuracy of about 1-2 digits. If we want to get around 6 digits of accuracy, we can use the commands:

In [ ]:
sol = solve(prob, Tsit5(),abstol=1e-8,reltol=1e-8)

Now we can see no visible difference against the true solution:

In [ ]:
plot(sol)
plot!(sol.t, t->1.0*exp(0.98t),lw=3,ls=:dash,label="True Solution!")

Notice that by decreasing the tolerance, the number of steps the solver had to take was `9` instead of the previous `5`. There is a trade off between accuracy and speed, and it is up to you to determine what is the right balance for your problem.

### Saveat

Another common option is to use `saveat` to make the solver save at specific time points. For example, if we want the solution at an even grid of 0.1 time unit, we would use the command:

In [ ]:
sol = solve(prob, Tsit5(), saveat=0.1)

Notice that when `saveat` is used the continuous output variables are no longer saved and thus `sol(t)`, the interpolation, is only first order. We can save at an uneven grid of points by passing a collection of values to `saveat`. For example:

In [ ]:
sol = solve(prob, Tsit5(),saveat=[0.2,0.7,0.9])

If we need to reduce the amount of saving, we can also turn off the continuous output directly via `dense=false`:

In [ ]:
sol = solve(prob, Tsit5(),dense=false)

and to turn off all intermediate saving we can use `save_everystep=false`:

In [ ]:
sol = solve(prob, Tsit5(),save_everystep=false)

If we want to solve and only save the final value, we can even set `save_start=false`.

In [ ]:
sol = solve(prob, Tsit5(),save_everystep=false,save_start = false)

Note that similarly on the other side there is `save_end=false`.

More advanced saving behaviors, such as saving functionals of the solution, are handled via the `SavingCallback` in the [Callback Library](http://docs.juliadiffeq.org/latest/features/callback_library.html#SavingCallback-1) which will be addressed later in the tutorial.

### Choosing Solver Algorithms

There is no best algorithm for numerically solving a differential equation. When you call `solve(prob)`, DifferentialEquations.jl (a larger package than OrdinaryDiffEq which we didn't install) makes a guess at a good algorithm for your problem, given the properties that you ask for (the tolerances, the saving information, etc.). However, in many cases you may want more direct control. However, such issues are beyond the scope of this workshop.

## Systems of ODEs: The Lorenz Equation

Now let's move to a system of ODEs. The [Lorenz equation](https://en.wikipedia.org/wiki/Lorenz_system) is the famous "butterfly attractor" that spawned chaos theory. It is defined by the system of ODEs:

$ \begin{align} \frac{dx}{dt} &= \sigma (y - x) \\ \frac{dy}{dt} &= x (\rho - z) -y \\ \frac{dz}{dt} &= xy - \beta z \end{align} $

To define a system of differential equations in DifferentialEquations.jl, we define our `f` as a vector function with a vector initial condition. Thus, for the vector `u = [x,y,z]'`, we have the derivative function:

In [ ]:
function lorenz!(du,u,p,t)
    σ,ρ,β = p
    du[1] = σ*(u[2]-u[1])
    du[2] = u[1]*(ρ-u[3]) - u[2]
    du[3] = u[1]*u[2] - β*u[3]
end

Notice here we used the in-place format which writes the output to the preallocated vector `du`. For systems of equations the in-place format is faster. We use the initial condition $u_0 = [1.0,0.0,0.0]$ as follows:

In [ ]:
u0 = [1.0,0.0,0.0]

Lastly, for this model we made use of the parameters `p`. We need to set this value in the `ODEProblem` as well. For our model we want to solve using the parameters $\sigma = 10$, $\rho = 28$, and $\beta = 8/3$, and thus we build the parameter collection:

In [ ]:
p = (10,28,8/3) # we could also make this an array, or any other sequence type!

Now we generate the `ODEProblem` type. In this case, since we have parameters, we add the parameter values to the end of the constructor call. Let's solve this on a time span of `t=0` to `t=100`:

In [ ]:
tspan = (0.0,100.0)
prob = ODEProblem(lorenz!,u0,tspan,p)

Now, just as before, we solve the problem:

In [ ]:
sol = solve(prob, Tsit5())

The same solution handling features apply to this case. Thus `sol.t` stores the time points and `sol.u` is an array storing the solution at the corresponding time points.

However, there are a few extra features which are good to know when dealing with systems of equations. First of all, `sol` also acts like an array. `sol[i]` returns the solution at the `i`th time point.

In [ ]:
println(sol.t[10], ",", sol[10])

Additionally, the solution acts like a matrix where `sol[j,i]` is the value of the `j`th variable at time `i`:

In [ ]:
sol[2,10]

We can get a real matrix by performing a conversion:

In [ ]:
A = Array(sol)

This is the same as sol, i.e. `sol[i,j] = A[i,j]`, but now it's a true matrix. Plotting will by default show the time series for each variable:

In [ ]:
plot(sol)

If we instead want to plot values against each other, we can use the `vars` command. Let's plot variable `1` against variable `2` against variable `3`:

In [ ]:
plot(sol,vars=(1,2,3), lw=1)

This is the classic Lorenz attractor plot, where the `x` axis is `u[1]`, the `y` axis is `u[2]`, and the `z` axis is `u[3]`. Note that the plot recipe by default uses the interpolation, but we can turn this off:

In [ ]:
plot(sol,vars=(1,2,3),denseplot=false)

## Cancer modelling

The use of Ordinary Differential Equations (ODEs) in cancer modelling has its roots in the broader application of mathematical models in biology and medicine, which began in earnest in the early 20th century. One of the early applications of ODEs was in population dynamics, a field that shares many conceptual similarities with the study of cellular populations in cancer.

The advent of the "Cancer by Numbers" concept by Laird in the 1960s (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2071101/pdf/brjcancer00492-0080.pdf) brought mathematical modelling into the field of cancer research. The mathematical models then were simple, but they laid the groundwork for more complex models. Laird's work used probability and simple mathematical models to explore cancer cell growth, survival, and mutation.

In the 1970s and 1980s, more complex ODE models began to appear. These models incorporated elements such as the cell cycle, the immune response, and the effects of chemotherapy. They began to portray cancer as a complex system, rather than a simple process of unchecked cell growth.

Since the turn of the 21st century, the development and application of ODE models in cancer research has exploded. This has been driven by several factors, including advances in computational power, the increasing availability of detailed biological data, and the recognition of the potential of mathematical models to improve our understanding of cancer and our ability to treat it.

Despite these advances, ODE modelling of cancer remains a very active field of research. Current challenges include the integration of multiscale models, which attempt to describe cancer from the molecular level up to the level of the whole organism, and the use of models to guide personalized treatment strategies.

# Classical Cancer Growth Models

## 1. Gompertzian Growth Model

The **Gompertzian growth model** is an exponential model of growth that was originally designed to describe human mortality rates but has been adapted to model tumour growth. In this model, the rate of tumour growth is fastest when the tumour is small, and slows as the tumour reaches its carrying capacity.

The Ordinary Differential Equation (ODE) for the Gompertzian growth model is:

$dc/dt = r * log(K/c) * c$

Where:

- `c` is the volume of cancer cells
- `t` is time
- `r` is the growth rate
- `K` is the carrying capacity of the environment

## 2. Logistic Growth Model

The **Logistic growth model** is a model of population growth where the growth rate decreases as the population nears its carrying capacity. This model assumes a symmetric growth pattern and is simpler than the Gompertzian model.

The ODE for the Logistic growth model is:

$dc/dt = r * c * (1 - c/K)$

In this equation, the parameters are the same as in the Gompertzian model.

## 3. von Bertalanffy Growth Model

The **von Bertalanffy growth model** was originally proposed to describe the growth of fish, but has been applied to cancer growth as well. It assumes that the rate of tumour growth is proportional to the two-thirds power of its size, reflecting the belief that the growth rate is determined by the surface area of the tumour rather than its volume.

The ODE for the von Bertalanffy growth model is:

$dc/dt = r * c^{2/3} - m * c$

Where:

- `r` is the growth constant
- `c` is the volume of cancer cells
- `m` is the mortality rate
- `t` is time

These models are all simplifications of the complex biological processes involved in cancer growth, but each can provide useful insights under different circumstances. The choice of model depends on the specific characteristics of the cancer being studied, the available data, and the questions being asked.



Let's now implement these models in julia and solve them for the same parameter set...

In [ ]:
r = 0.1 # growth rate
K = 1.0 # carrying capacity
c0 = 0.01 # initial condition

gompertzian(u,p,t) = r * log(K/u) * u

tspan = (0.0, 100.0)
prob = ODEProblem(gompertzian, c0, tspan)
sol1 = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)

plot(sol1, ylabel="Volume of Cancer Cells", xlabel="Time", title="Gompertzian Growth Model", legend=false)

In [ ]:
r = 0.1 # growth rate
K = 1.0 # carrying capacity
c0 = 0.01 # initial condition

logistic(u,p,t) = r * u * (1 - u/K)

prob = ODEProblem(logistic, c0, tspan)
sol2 = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)

plot(sol2, ylabel="Volume of Cancer Cells", xlabel="Time", title="Logistic Growth Model", legend=false)


In [ ]:
r = 0.1 # growth constant
m = 0.01 # mortality rate
c0 = 0.01 # initial condition

von_bertalanffy(u,p,t) = r * u^(2/3) - m * u

prob = ODEProblem(von_bertalanffy, c0, tspan)
sol3 = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)

plot(sol3, ylabel="Volume of Cancer Cells", xlabel="Time", title="Von Bertalanffy Growth Model", legend=false)


Do you think the dynamics are similar? Perhaps easier to see if we plot them on the same plot with the y-axis on the log-scale

In [ ]:
plot(sol1,yscale=:log10,label="Logistic growth",xlabel="Time",ylabel="Volume of Cancer Cells")
plot!(sol2,label="Logistic growth",xlabel="Time",ylabel="Volume of Cancer Cells")
plot!(sol3,label="Von Bertanlanffy",xlabel="Time",ylabel="Volume of Cancer Cells")

If we use the same parameters for all three models (Gompertzian, Logistic, and von Bertalanffy), we can observe some general characteristics about their dynamics:

Logistic Growth Model: This model starts with exponential growth which then slows down and asymptotically approaches the carrying capacity, K. The growth is symmetric, meaning the rate of growth is the same up to the inflection point as it is beyond that.

Gompertzian Growth Model: The Gompertzian model also exhibits a sigmoidal growth pattern similar to the logistic model, but its growth is not symmetric. The growth slows down more rapidly than in the logistic model, and the growth rate continually declines as the population size approaches the carrying capacity, K. This model might better reflect tumour growth in some cases, as it captures the observation that tumours often slow down their growth rate as they increase in size.

von Bertalanffy Growth Model: The von Bertalanffy model, like the Gompertzian, also assumes that the growth rate declines as the population increases. But unlike the other two models, the von Bertalanffy model's rate of growth is proportional to the two-thirds power of its size. This can make the von Bertalanffy model more accurate for systems where growth is more influenced by the surface area (which generally scales with the 2/3 power of volume) than by the volume itself.

All three models will show that growth initially starts rapidly and then slows as it approaches the carrying capacity. However, the rate at which growth slows and the symmetry of the growth curve can vary between the models. Remember, these models are simplifications of the complex biological processes involved in cancer growth. The best model to use will depend on the specific characteristics of the tumour, the available data, and the questions being asked in the research study.

Given the below data, try fitting each of the models analytic solution using LsqFit (which we encountered in the first covid-19 modelling notebook) and compute the Akaike information criterion or Bayesian information criterion for each model. Try to figure out which of the three models is best at capturing the data!

In [ ]:
tumour_data = [0.004271329261712347,0.012090299258236367,0.06047945122837756,0.07955439331530145,0.13488275672591388
,0.1794705672612774,0.24403857722753283,0.3281076427474533,0.4019987281192025,0.4659433690807729,0.5372763041777083]
tdata = collect(0.0:2.0:20.0)

Hint: Use the following analytic solutions and look up Akaike Information Criterion and Bayesian information criterion...

In [ ]:
using LsqFit

# models in the form expected by LsqFit.curve_fit
model_logistic(t,p) = p[1] .* exp.(p[2] .* t) ./ (1 .+ p[1] .* (exp.(p[2] .* t) .- 1) ./ p[3])
model_gompertzian(t,p) = p[3] .* exp.(log(p[1] ./ p[3]) .* exp.(-p[2] .* t))
model_vonbertalanffy(t,p) = abs(p[2]) .* (1.0 .- exp.(-abs(p[1]) .* t)) .^ (2.0/3.0)

# initial guess
p0 = [c0, r, K]
p1 = [r, K]
# fit the models
fit_logistic = curve_fit(model_logistic, tdata, tumour_data, p0)
fit_gompertzian = curve_fit(model_gompertzian, tdata, tumour_data, p0)
fit_vonbertalanffy = curve_fit(model_vonbertalanffy, tdata, tumour_data, p1)

# residuals (RSS)
resid_logistic = sum(abs2, fit_logistic.resid)
resid_gompertzian = sum(abs2, fit_gompertzian.resid)
resid_vonbertalanffy = sum(abs2, fit_vonbertalanffy.resid)